In [15]:
import pandas as pd
import os
import pickle
import subprocess

import numpy as np
import pandas as pd
from flask import Flask, jsonify, request
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('./data/online_gaming_behavior_dataset.csv')
data

,PlayerID,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked,EngagementLevel
0,9000,43,Male,Other,Strategy,16.271119,0,Medium,6,108,79,25,Medium
1,9001,29,Female,USA,Strategy,5.525961,0,Medium,5,144,11,10,Medium
2,9002,22,Female,USA,Sports,8.223755,0,Easy,16,142,35,41,High
3,9003,35,Male,USA,Action,5.265351,1,Easy,9,85,57,47,Medium
4,9004,33,Male,Europe,Action,15.531945,0,Medium,2,131,95,37,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,10095,35,Male,Europe,Simulation,5.545389,0,Easy,7,133,42,25,Medium
1096,10096,36,Male,USA,RPG,4.382727,1,Easy,5,169,44,20,Medium
1097,10097,21,Female,Europe,Action,15.578916,0,Medium,18,178,42,44,Low
1098,10098,28,Male,USA,Strategy,12.814229,0,Easy,2,33,18,39,Low


In [4]:
model = pickle.load(open("model.pkl", "rb"))

In [9]:
num_col = ['SessionsPerWeek', 'AvgSessionDurationMinutes', 'AchievementsUnlocked']
target = 'EngagementLevel'

train_set, test_set = train_test_split(data, test_size=0.2, stratify=data[target], random_state=42)

X_train = train_set[num_col]
X_test = test_set[num_col]

y_train = train_set[target]
y_test = test_set[target]

In [ ]:
model.

In [13]:
balanced_accuracy_score(y_test, model.predict(X_test))

ValueError: Mix of label input types (string and number)

In [17]:
data = pd.read_csv("./data/online_gaming_behavior_dataset.csv")



#instanciamos lista de columnas con las que entrenamos
num_col = ['SessionsPerWeek', 'AvgSessionDurationMinutes', 'AchievementsUnlocked']

#instanciamos el target
target = 'EngagementLevel'


#dividimos en train y test
train_set, test_set = train_test_split(data, test_size=0.2, stratify=data[target], random_state=42)

X_train = train_set[num_col]
X_test = test_set[num_col]

y_train = train_set[target]
y_test = test_set[target]


#mapearemos 'y' para entrenar
labels = {'Low':0,
          'Medium': 1,
          'High': 2}
y_train_encoded = y_train.map(labels)
y_test_encoded = y_test.map(labels)

#Preprocessing
num_pipe = Pipeline([
    ('mmscaler', MinMaxScaler())
])

preprocessing = ColumnTransformer([
    ('normalize', num_pipe, num_col)],
    remainder='passthrough').set_output(transform= 'pandas')


#Entrenamiento
logreg_pipe = Pipeline([
    ('preprocessing', preprocessing),
    ('modelo', LogisticRegression(class_weight='balanced', C=1.0, max_iter=50, 
                                  solver='saga', random_state=42))])

logreg_pipe.fit(X_train, y_train_encoded)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('normalize',
                                                  Pipeline(steps=[('mmscaler',
                                                                   MinMaxScaler())]),
                                                  ['SessionsPerWeek',
                                                   'AvgSessionDurationMinutes',
                                                   'AchievementsUnlocked'])])),
                ('modelo',
                 LogisticRegression(class_weight='balanced', max_iter=50,
                                    random_state=42, solver='saga'))])

In [18]:
y_pred_logreg= logreg_pipe.predict(X_test) 
prediccion_lg = balanced_accuracy_score(y_test_encoded, y_pred_logreg)
prediccion_lg

0.7730286954424885

In [33]:
type(y_test_encoded)

pandas.core.series.Series

In [41]:
X_test

,SessionsPerWeek,AvgSessionDurationMinutes,AchievementsUnlocked
887,15,17,12
936,11,59,17
660,4,52,42
444,10,74,5
42,15,68,39
...,...,...,...
297,12,79,5
408,18,169,49
341,9,104,45
161,11,90,5


In [38]:
type(y_true)

pandas.core.series.Series

In [40]:
prediction = logreg_pipe.predict(y_true)
prediction

ValueError: X does not contain any features, but ColumnTransformer is expecting 3 features

In [43]:

api_get = {'SessionsPerWeek': 54,
           'AchievementsUnlocked': 77,
           'AvgSessionDurationMinutes': 88}
df_get = pd.DataFrame(api_get, index=[0])
df_get

,SessionsPerWeek,AchievementsUnlocked,AvgSessionDurationMinutes
0,54,77,88


In [ ]:
prediction = logreg_pipe.predict(df_get)
prediction[0]


2